In [5]:
import heapq
import math
from myHeap import MyHeap

def vaild(Ai,M):
    if (len(Ai)==0):
        return False
    for i in Ai:
        if (2*M+1-i) in Ai:
            return False
    return True

def PerComputeScores(W,M):
    scores=dict()
    for j in range(1,2*M+1):
        if j>=1 and j <=M:
            score=(j*(j+1)*math.pow(W,2))/(4*(M+1)*(M+2))
        else:
            score=math.pow(W,2)*(1-(2*M+1-j)/(M+1)+((2*M+1-j)*(2*M+2-j))/(4*(M+1)*(M+2)))
        scores[j]=score
    return scores

def PerComputeProbingSequence(W,M,T):
    """W：段长    M：哈希函数h个数    T：扰动向量个数"""
    scores=PerComputeScores(W,M)
    myheap=MyHeap(scores)
    A0=[1]
    myheap.initpush(A0)
    AT=list()
    for i in range(1,1000):
        Ai=list()
        while len(myheap.data)>0 and not vaild(Ai,M) :
            Ai=myheap.pop()
            if max(Ai)>2*M-1:
                break
            temp1=temp2=Ai[:]
            myheap.shift(temp1)
            myheap.expand(temp2)
        if vaild(Ai,M):
            AT.append(Ai)
    return AT[:T]


AT=PerComputeProbingSequence(30,3,1000)
print(len(AT))

26


In [1]:
import numpy as np
import math
import random
import heapq
import time
import matplotlib.pyplot as plt
from myHeap import MyHeap

def vaild(Ai,M):
    if (len(Ai)==0):
        return False
    for i in Ai:
        if (2*M+1-i) in Ai:
            return False
    return True

def PerComputeScores(W,M):
    scores=dict()
    for j in range(1,2*M+1):
        if j>=1 and j <=M:
            score=(j*(j+1)*math.pow(W,2))/(4*(M+1)*(M+2))
        else:
            score=math.pow(W,2)*(1-(2*M+1-j)/(M+1)+((2*M+1-j)*(2*M+2-j))/(4*(M+1)*(M+2)))
        scores[j]=score
    return scores

def PerComputeProbingSequence(W,M,T):
    """W：段长    M：哈希函数h个数    T：扰动向量个数"""
    scores=PerComputeScores(W,M)
    myheap=MyHeap(scores)
    A0=[1]
    myheap.initpush(A0)
    AT=list()
    for i in range(1,1000):
        Ai=list()
        while len(myheap.data)>0 and not vaild(Ai,M) :
            Ai=myheap.pop()
            if max(Ai)>2*M-1:
                break
            temp1=temp2=Ai[:]
            myheap.shift(temp1)
            myheap.expand(temp2)
        if vaild(Ai,M):
            AT.append(Ai)
    return AT[:T]

def E2Distance(x,y):  
    """L2范式下欧几里得空间距离计算"""
    return math.sqrt(sum(map(lambda i,j:math.pow(i-j,2),x,y)))

def kClosestSort(points,k,origin):
    """返回离点origin在L1范式下欧几里得空间距离最靠近的k个点（堆排序） 时间复杂度Θ(nlogn)"""
    return heapq.nsmallest(k, points, lambda p:E2Distance(p,origin) )

def E2LSH_Init(L,k,T=100,r=30):
    """ k:线段编号长度  L:哈希表的数量(实线数量) r:线段长度 """
    data=np.load('E2LSH_DataSet.npy')
    n=data.shape[0]
    d=data.shape[1]
    hashtable=dict()
    #for i in range(L):
        #a_values=np.random.normal(0,1,size=(k,d))
        #b_values=np.random.uniform(0,r,k)
        #hashtable[i]=(a_values,b_values)
        #hashtable[i]=a_values
    hashtable=np.load('HashFunction_l='+str(L)+'.npy').item()
    lines=dict()
    for i in range(L):
        line=dict()
        for j in range(n):
            hashcode=list()
            for v in range(k):
                code=int(math.floor((np.inner(hashtable[i][0][v],data[j])+hashtable[i][1][v])/r))
                hashcode.append(code)
            hashcode=tuple(hashcode)
            if hashcode not in line.keys():
                line[hashcode]=set()
            line[hashcode].add(tuple(data[j]))
        lines[i]=line
    #np.save('HashFunction_l='+str(L),hashtable) 
    AT=PerComputeProbingSequence(r,k,T)
    return lines,AT


def E2LSH_Inquire(L,lines,AT,r=30,K=20):
    quires=np.load('E2LSH_Quires.npy')
    hashtable=np.load('HashFunction_l='+str(L)+'.npy').item()
    L=len(hashtable)
    k=len(hashtable[0][0])
    qnum=len(quires)
    result=dict()
    for q in range(qnum):
        S=set()
        for i in range(L):
            hashcodes=list()
            hashcode=list()
            xi=dict()
            for j in range(k):
                code=int(math.floor((np.inner(hashtable[i][0][j],quires[q])+hashtable[i][1][j])/r))
                hashcode.append(code)
                xi[(j,-1)]=(np.inner(hashtable[i][0][j],quires[q])+hashtable[i][1][j])-code*r
                xi[(j,1)]=r-xi[(j,-1)]
            hashcodes.append(tuple(hashcode))
            pai=sorted(xi,key=xi.get) #按值排序
            for v in AT:
                temp_code=hashcode[:]
                for n in v:
                    temp_code[pai[n-1][0]]+=pai[n-1][1]
                hashcodes.append(tuple(temp_code))
            for hashcode in hashcodes:
                if hashcode in lines[i].keys():
                    S=S|lines[i][hashcode]         
        S=list(S)
        SK=list()
        if len(S)>K:
            SK=kClosestSort(S,K,quires[q])
        else:
            SK=S
        result[q]=SK
    return result,len(AT)*L

def Brute_ForceLinearSearch(K=20):
    """线性暴力搜索k近点"""
    data=np.load('E2LSH_DataSet.npy')
    quires=np.load('E2LSH_Quires.npy')
    data_temp=data.tolist()
    data=list()
    for i in data_temp:
        data.append(tuple(i))
    qnum=len(quires)
    real=dict()
    for q in range(qnum):
        q_real=kClosestSort(data,K,quires[q])
        real[q]=q_real
    return real

def Recall(result,real):
    quires=np.load('E2LSH_Quires.npy')
    qnum=len(real)
    temp1=0
    temp2=0
    for q in range(qnum):
        K=len(real[q])
        kdis=E2Distance(quires[q],real[q][len(real[q])-1])
        for point in result[q]:
            if E2Distance(quires[q],point)<=kdis:
                temp1+=1
        #temp1+=len(list(set(result[q]).intersection(set(real[q]))))
        temp2+=K
    recall=temp1/temp2
    return recall

def plot_trendline(x, y, n):
    plt.title("20-ANNS  n=100000,d=10,k=3,r=30,|Q|=100", fontsize=14)
    plt.xlabel("Recall", fontsize=14)
    plt.ylabel("Number of hash tables", fontsize=14)
    plt.scatter(x, y,c='k',s=10)
    parameter = np.polyfit(x, y, n) # n=1为一次函数，返回函数参数
    f = np.poly1d(parameter) # 拼接方程
    plt.plot(x, f(x))
    #plt.axis([0, 1.1, 1, 128])
    plt.savefig('111.png', bbox_inches='tight')
    return
'''
temp=list()
T=list()
real=Brute_ForceLinearSearch()
for i in range (1,8):
    lines,AT=E2LSH_Init(8,3,4*i)
    result,probesum=E2LSH_Inquire(8,lines,AT)
    recall=Recall(result,real)
    temp.append(recall)
    T.append(probesum)
print(temp)
print(T)
plot_trendline(temp,T,7)
'''
temp=list()
T=list()
real=Brute_ForceLinearSearch()
for i in range (1,8):
    lines,AT=E2LSH_Init(int(math.pow(2,i)),3)
    result,probesum=E2LSH_Inquire(int(math.pow(2,i)),lines,AT)
    recall=Recall(result,real)
    temp.append(recall)
    T.append(int(math.pow(2,i)))
print(temp)
print(T)
plot_trendline(temp,T,7)

[0.6345, 0.761, 0.9875, 0.999, 1.0, 1.0, 1.0]
[2, 4, 8, 16, 32, 64, 128]


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:158: RankWarning: Polyfit may be poorly conditioned
